##  A tour of using the revised DNAStack WES Server

This will work with Thousand Genomes bam files. Even though the Thousand Genomes bam files are publically available care was taken in the following not to show the whole contents of the run details returned by the WES server. The response contains signed URLs. In other cases that would allow access to controlled access data - albeit for a limited time.

This function shows some details of the run, but not the possibly sensitive stuff.

In [6]:
def getMinimalRunLog(wesClient, run_id):
    rundetails = wesClient.GetRunLog(run_id, verbose=True)
    print(rundetails)
    print(rundetails['run_id'])
    print(rundetails['state'])
    print(rundetails['outputs'])

We can now use that function to minimal details of a run submitted via FASPScript4.py. That script submits a workflow to run an MD5 checksum on a Thousand Genomes bam file. It's output should just be that checksum.

In [9]:
from fasp.workflow import DNAStackWESClient
wesClient = DNAStackWESClient('~/.keys/dnastack_wes_credentials.json')
run_id = '1d764abd-5c8f-4c89-ad2b-26a43b2025f1'
getMinimalRunLog(wesClient, run_id)

Get request sent to: https://workspaces-wes.prod.dnastack.com/ga4gh/wes/v1/runs/1d764abd-5c8f-4c89-ad2b-26a43b2025f1
<Response [403]>
None


TypeError: 'NoneType' object is not subscriptable

That indicates an empty response. Though the task appeared to complete successfully we have no result. How can we check what happened? Links to stdout and stderr are available in the response.

In [3]:
rundetails = wesClient.GetRunLog(run_id)
log = rundetails['task_logs'][0]
for stdx in ['stdout','stderr']:
    print('{} is at {}\n'.format(stdx, log[stdx]))


<Response [404]>


TypeError: 'NoneType' object is not subscriptable

Following those links downloads empty files. That is the same as the previous version of the server. It was suggested that DRS would be a suitable way to make the files available.

The absence of the files means we don't have access to information that would be useful for troubleshooting. The guess is that it is the same problem as previously.

Trying another workflow. 

The function runGWASWorkflowTest encapsulates the GWAS Workflow from the 2020 GA4GH Plenary. It runs it on files accessed via Google Cloud URIs rather than DRS. This lets us check a known workflow.

In [9]:
wesClient.runGWASWorkflowTest()

'eaf4ff6a-e229-47b9-9e4c-558dbb2c16be'

In [12]:
getMinimalRunLog(wesClient, 'eaf4ff6a-e229-47b9-9e4c-558dbb2c16be')

eaf4ff6a-e229-47b9-9e4c-558dbb2c16be
RUNNING
{}


In this case the run has produced some output files which look promising. How we access those files is still an open question. See discussion elsewhere about whether a companion DRS server, or even a DRS server accessed via the WES server and accessed under the same credentials would be a useful addition to WES.